In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

# Add /src modules to path
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Import modules
from src.utils import MySQLConnector, haversine_np
from itertools import combinations
import pandas as pd
import numpy as np

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [4]:
# Connect to MySQL database

connector = MySQLConnector()
cursor = connector.cursor
db_connection = connector.db_connection

Connected to: 8.1.0
You are connected to the database: ('mysql',)
-----------------------------------------------



### Task 1
How many users, activities and trackpoints are there in the dataset?

In [111]:
query = """
    SELECT 
    (SELECT COUNT(*) FROM UserTable) AS user_count,
    (SELECT COUNT(*) FROM ActivityTable) AS activity_count,
    (SELECT COUNT(*) FROM TrackPointTable) AS trackpoint_count;
"""

cursor.execute(query)
result = cursor.fetchall()
result_df = pd.DataFrame(result, columns=['user_count', 'activity_count', 'trackpoint_count'])
result_df

,user_count,activity_count,trackpoint_count
0,182,14715,3851557


### Task 2
Find the average, maximum and minimum number of trackpoints per user.

In [112]:
query = """
    SELECT
        AVG(trackpoint_count) AS average,
        MAX(trackpoint_count) AS max,
        MIN(trackpoint_count) AS min
    FROM 
    (
        SELECT
            UserTable.Id AS user_id,
            COUNT(TrackPointTable.Id) AS trackpoint_count
        FROM UserTable
        LEFT JOIN ActivityTable ON UserTable.Id = ActivityTable.user_id
        LEFT JOIN TrackPointTable ON ActivityTable.Id = TrackPointTable.activity_id
        GROUP BY UserTable.Id
    ) AS UserTrackpointCount;
"""


cursor.execute(query)
result = cursor.fetchall()
result_df = pd.DataFrame(result, columns=['average', 'max', 'min'])
result_df['average'] = result_df['average'].apply(float).round(1)
result_df


,average,max,min
0,21162.40,603380,0


### Task 3
Find the top 15 users with the highest number of activities.

In [113]:
query = """
    SELECT user_id, COUNT(*) AS activity_count
    FROM ActivityTable
    GROUP BY user_id
    ORDER BY activity_count DESC
    LIMIT 15; 
"""

cursor.execute(query)
result = cursor.fetchall()

result_df = pd.DataFrame(result, columns=['user_id', 'number_of_activities'])
result_df

,user_id,number_of_activities
0,163,3182
1,085,1298
2,153,1123
3,068,969
4,167,944
5,128,937
6,062,782
7,075,509
8,126,468
9,010,434


### Task 4
Find all users who have taken a bus.

In [12]:
query = """
    SELECT DISTINCT user_id
    FROM ActivityTable
    WHERE transportation_mode = 'bus';
"""

cursor.execute(query)
result = cursor.fetchall()

print(result)

[('010',), ('020',), ('052',), ('053',), ('058',), ('062',), ('064',), ('065',), ('067',), ('068',), ('069',), ('073',), ('075',), ('078',), ('080',), ('081',), ('082',), ('084',), ('085',), ('091',), ('092',), ('096',), ('098',), ('100',), ('101',), ('102',), ('104',), ('105',), ('108',), ('110',), ('111',), ('112',), ('114',), ('125',), ('126',), ('128',), ('129',), ('138',), ('139',), ('141',), ('147',), ('153',), ('154',), ('161',), ('163',), ('167',), ('174',), ('175',), ('179',)]


### Task 5
List the top 10 users by their amount of different transportation modes.

In [117]:
query = """
    SELECT user_id, COUNT(DISTINCT transportation_mode) AS distinct_transportation_mode_count
    FROM ActivityTable
    GROUP BY user_id
    ORDER BY distinct_transportation_mode_count DESC
    LIMIT 10;
"""

cursor.execute(query)
result = cursor.fetchall()

result_df = pd.DataFrame(result, columns=['user_id', 'num_transportation_modes'])
result_df

,user_id,num_transportation_modes
0,128,10
1,075,9
2,062,9
3,163,9
4,167,8
5,153,8
6,084,8
7,126,8
8,020,7
9,085,7


### Task 6
Find activities that are registered multiple times. You should find the query even
if it gives zero result.

In [14]:
query = """
    SELECT user_id, transportation_mode, start_date_time, end_date_time, COUNT(*)
    FROM ActivityTable
    GROUP BY user_id, transportation_mode, start_date_time, end_date_time
    HAVING COUNT(*) > 1;
"""

cursor.execute(query)
result = cursor.fetchall()

result

[]

### Task 7

#### a) 
Find the number of users that have started an activity in one day and ended
the activity the next day.

In [118]:
query = """
    SELECT 
        user_id, 
        transportation_mode, 
        TIMESTAMPDIFF(MINUTE, start_date_time, end_date_time) AS duration_minutes
    FROM 
        ActivityTable
    WHERE 
        DATEDIFF(end_date_time, start_date_time) = 1;
"""


cursor.execute(query)
result = cursor.fetchall()

result_df = pd.DataFrame(result, columns=['user_id', 'transportation mode', 'duration (min)'])
print("Number of users", result_df["user_id"].unique().shape[0])


Number of users 38


#### b)
List the transportation mode, user id and duration for these activities.

In [119]:
result_df

,user_id,transportation mode,duration (min)
0,098,taxi,730
1,106,car,1439
2,153,taxi,556
3,021,car,360
4,076,car,538
...,...,...,...
398,163,car,1079
399,163,car,1079
400,163,car,1019
401,163,car,1019


### Task 8
Find the number of users which have been close to each other in time and space.
Close is defined as the same space (50 meters) and for the same half minute (30
seconds)

In [18]:
query = """
    SELECT id, user_id, start_date_time, end_date_time FROM ActivityTable
"""
cursor.execute(query)
activity_result = cursor.fetchall()
activity_result_df = pd.DataFrame(activity_result, columns=['id', 'user_id', 'start_date_time', 'end_date_time'])

In [19]:
query = """
    SELECT activity_id, lat, lon, altitude, date_time FROM TrackPointTable
"""
cursor.execute(query)
result = cursor.fetchall()

In [20]:
result_df = pd.DataFrame(result, columns=["activity_id", "lat", "lon", "altitude", "date_time"])

# Turn date_time into seconds
result_df["date_time"] = pd.to_datetime(result_df["date_time"]).astype(int) / 10**9

max_date = result_df["date_time"].max()
min_date = result_df["date_time"].min()

In [21]:
from tqdm import tqdm

# Define num of batches
NUM_BATCHES = 20000
# Define step size
step = (max_date - min_date) / NUM_BATCHES

pbar = tqdm(range(0, NUM_BATCHES))

result = []
for i in pbar:
    # Filter the batches, with 60 seconds overlap
    tmp = result_df[
        (result_df["date_time"] > i * step + min_date)
        & (result_df["date_time"] < (i + 1) * step + 60 + min_date)
    ]
    pbar.set_postfix({"batch": i, "shape": tmp.shape})

    # If the batch is empty, continue
    if tmp.shape[0] == 0:
        continue
    
    # Get user IDs from ActivityTable
    tmp = pd.merge(tmp, activity_result_df, left_on="activity_id", right_on="id")
    tmp["date_time"] = pd.to_datetime(tmp["date_time"] * 10**9)

    # Find unique combinations of user ids
    user_ids = tmp["user_id"].unique()
    unique_combinations = []
    for combo in combinations(user_ids, 2):
        if combo[::-1] not in unique_combinations:
            unique_combinations.append(combo)

    # Loop through user combinations
    for combo in unique_combinations:
        if (combo[0], combo[1]) in result or (combo[1], combo[0]) in result:
            continue
        
        tmp_u1 = tmp[tmp["user_id"] == combo[0]]
        tmp_u2 = tmp[(tmp["user_id"] == combo[1])]

        # Create a cross product df of the two users
        combo_df = tmp_u1.merge(tmp_u2, how="cross")

        # Filter on altitude and time difference
        filtered_df = combo_df[
            (np.abs(combo_df.altitude_x - combo_df.altitude_y) < 5)
            & (
                abs(combo_df["date_time_x"] - combo_df["date_time_y"])
                <= pd.Timedelta(seconds=30)
            )
        ]

        # If there are any rows in the filtered df, calculate the distance
        if filtered_df.shape[0] > 0:
            distances = haversine_np(
                filtered_df["lon_x"],
                filtered_df["lat_x"],
                filtered_df["lon_y"],
                filtered_df["lat_y"],
            ) * 1000

            # Check if any distance is below 5 meters
            has_distance_below_5m = any(distances < 5)

            if has_distance_below_5m:
                result.append((combo[0], combo[1]))

100%|██████████| 20000/20000 [05:02<00:00, 66.15it/s, batch=2e+4, shape=(352, 5)]   


In [22]:
print("UserID combinations:")
print('number of user pairs within 50 meters for the same half minute: ' + str(len(result))) 
result

UserID combinations:
number of user pairs within 50 meters for the same half minute: 35


[('111', '128'),
 ('139', '108'),
 ('175', '163'),
 ('175', '056'),
 ('175', '114'),
 ('163', '056'),
 ('174', '101'),
 ('167', '126'),
 ('167', '163'),
 ('163', '089'),
 ('144', '089'),
 ('153', '163'),
 ('128', '153'),
 ('081', '125'),
 ('128', '163'),
 ('167', '129'),
 ('167', '112'),
 ('153', '081'),
 ('153', '125'),
 ('167', '153'),
 ('167', '073'),
 ('153', '073'),
 ('153', '112'),
 ('078', '112'),
 ('163', '112'),
 ('167', '069'),
 ('129', '069'),
 ('073', '112'),
 ('167', '096'),
 ('084', '085'),
 ('078', '073'),
 ('167', '078'),
 ('062', '064'),
 ('128', '179'),
 ('102', '020')]

### Task 9

Find the top 15 users who have gained the most altitude meters.

In [98]:
query = """
    SELECT activity_id, altitude, date_time FROM TrackPointTable
"""
cursor.execute(query)
result = cursor.fetchall()

In [99]:
query = """
    SELECT id, user_id FROM ActivityTable
"""
cursor.execute(query)
user_activity = cursor.fetchall()
user_activity_df = pd.DataFrame(user_activity, columns=["activity_id", "user_id"])

In [100]:
result_df = pd.DataFrame(result, columns=["activity_id", "altitude", "date_time"])

result_df["altitude_t-1"] = result_df.groupby("activity_id")["altitude"].shift(1)
result_df["altitude_gain"] = result_df["altitude"] - result_df["altitude_t-1"]
result_df = result_df[~result_df["altitude_gain"].isna()]
result_df = result_df[result_df["altitude_gain"] > 0]
diff_df = result_df.copy().merge(user_activity_df, on="activity_id")

In [101]:
diff_df["altitude_gain"] = diff_df["altitude_gain"].apply(float).round(2)

diff_df.groupby("user_id")["altitude_gain"].sum().reset_index().sort_values(
    "altitude_gain", ascending=False
).head(15)

,user_id,altitude_gain
41,128,873880.26
19,085,852330.00
7,062,500368.40
18,084,435454.00
52,167,426178.44
48,153,370663.51
2,052,268203.00
40,126,180439.89
51,163,164277.28
0,010,163642.00


### Task 10
Find the users that have traveled the longest total distance in one day for each
transportation mode.

In [5]:
# Fetch trackpoint data
query = """
    SELECT activity_id, lat, lon, date_time FROM TrackPointTable
"""
cursor.execute(query)
track_points = cursor.fetchall()
track_points_df = pd.DataFrame(
    track_points, columns=["activity_id", "lat", "lon", "date_time"]
)

# Fetch user activity data
query = """
    SELECT id, user_id, transportation_mode FROM ActivityTable
"""
cursor.execute(query)
user_activity = cursor.fetchall()
user_activity_df = pd.DataFrame(user_activity, columns=["activity_id", "user_id", "transportation_mode"])

In [6]:
# Merge trackpoint data with user activity
user_track_points_df = track_points_df.merge(user_activity_df, on="activity_id")

# Create shifted_points dataframe with shifted lat and lon values
shifted_points = user_track_points_df.copy()
shifted_points["date"] = pd.to_datetime(shifted_points["date_time"]).dt.date
shifted_points = shifted_points.groupby(["user_id", "activity_id", "date"])[
    ["user_id", "activity_id", "date", "lat", "lon"]
].shift(1)
missing_indices = shifted_points[shifted_points["user_id"].isna()].index
shifted_points = shifted_points.dropna()
user_track_points_df = user_track_points_df.drop(missing_indices)

In [7]:
# Calculate haversine distances
distances = haversine_np(
    user_track_points_df["lon"], user_track_points_df["lat"],
    shifted_points["lon"], shifted_points["lat"],
)

In [8]:
result_df = user_track_points_df.copy()
result_df["date"] = pd.to_datetime(result_df["date_time"]).dt.date
result_df["distance (km)"] = distances


daily_distance = result_df.groupby(['user_id', 'date', 'transportation_mode'])['distance (km)'].sum().reset_index()
idx = daily_distance.groupby('transportation_mode')['distance (km)'].idxmax()
top_travelers = daily_distance.loc[idx]
top_travelers = top_travelers.sort_values(by='distance (km)', ascending=False)
final_result = top_travelers[['user_id', 'transportation_mode', 'distance (km)']]
final_result.loc[:, 'distance (km)'] = final_result['distance (km)'].apply(float).round(2)
final_result


,user_id,transportation_mode,distance (km)
3375,128,airplane,2529.99
2,010,train,1545.35
302,021,car,1176.27
3607,153,taxi,1107.43
3333,128,bus,214.97
3610,153,walk,192.08
447,052,subway,150.80
3247,128,boat,134.25
2470,111,bike,121.18
3029,128,run,3.84


### Task 11
Find all users who have invalid activities, and the number of invalid activities per 
user.

 An invalid activity is defined as an activity with consecutive trackpoints
where the timestamps deviate with at least 5 minutes.

In [36]:
# Fetch track points data
query_track_points = """
    SELECT activity_id, date_time FROM TrackPointTable
"""
cursor.execute(query_track_points)
track_points = cursor.fetchall()
track_points_df = pd.DataFrame(track_points, columns=["activity_id", "date_time"])

# Fetch user activity data
query_user_activity = """
    SELECT id, user_id FROM ActivityTable
"""
cursor.execute(query_user_activity)
user_activity = cursor.fetchall()
user_activity_df = pd.DataFrame(user_activity, columns=["activity_id", "user_id"])

In [37]:
# Merge and process date_time
user_track_points_df = track_points_df.merge(user_activity_df, on="activity_id")
user_track_points_df["date_time"] = pd.to_datetime(user_track_points_df["date_time"])

# Shift and filter invalid activities
user_track_points_df["date_time_t-1"] = user_track_points_df.groupby("activity_id")["date_time"].shift(1)
user_track_points_df = user_track_points_df.dropna()

invalid_activities = user_track_points_df[
    (user_track_points_df["date_time"] - user_track_points_df["date_time_t-1"])
    >= pd.Timedelta(minutes=5)
][["activity_id", "user_id"]]

In [40]:
invalid_activity_counts = invalid_activities.drop_duplicates().groupby("user_id").count().reset_index()
invalid_activity_counts.values.tolist()

[['010', 53],
 ['020', 26],
 ['021', 12],
 ['052', 117],
 ['053', 8],
 ['056', 3],
 ['058', 6],
 ['060', 1],
 ['062', 184],
 ['064', 9],
 ['065', 23],
 ['067', 32],
 ['068', 128],
 ['069', 7],
 ['073', 7],
 ['075', 4],
 ['076', 7],
 ['078', 7],
 ['080', 4],
 ['081', 11],
 ['082', 15],
 ['084', 155],
 ['085', 219],
 ['086', 6],
 ['087', 2],
 ['089', 17],
 ['091', 12],
 ['092', 4],
 ['096', 10],
 ['097', 14],
 ['098', 7],
 ['101', 1],
 ['102', 9],
 ['105', 14],
 ['106', 3],
 ['107', 1],
 ['108', 7],
 ['111', 10],
 ['112', 31],
 ['114', 2],
 ['115', 45],
 ['117', 1],
 ['125', 13],
 ['126', 97],
 ['128', 238],
 ['129', 5],
 ['138', 2],
 ['139', 12],
 ['141', 8],
 ['144', 5],
 ['153', 148],
 ['154', 10],
 ['161', 2],
 ['163', 79],
 ['167', 135],
 ['170', 2],
 ['174', 2],
 ['175', 5],
 ['179', 53]]

### Task 12
Find all users who have registered transportation_mode and their most used
transportation_mode.

In [120]:
query = """
    SELECT
    user_id,
    transportation_mode
    FROM (
        SELECT
            user_id,
            transportation_mode,
            COUNT(*) AS mode_count,
            RANK() OVER (PARTITION BY user_id ORDER BY COUNT(*) DESC) AS rnk
        FROM ActivityTable
        WHERE transportation_mode IS NOT NULL
        GROUP BY user_id, transportation_mode
    ) AS subquery
    WHERE rnk = 1;
"""
cursor.execute(query)
track_points = cursor.fetchall()
track_points_df = pd.DataFrame(
    track_points, columns=["user_id", "transporation_mode"]
)

track_points_df

,user_id,transporation_mode
0,010,walk
1,020,bike
2,021,car
3,052,bus
4,053,walk
...,...,...
64,167,walk
65,170,walk
66,174,car
67,175,walk


In [ ]:
connector.close_connection()


-----------------------------------------------
Connection to None is closed
